In [1]:
pip install kagglehub


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("shayanfazeli/heartbeat")

print("Path to dataset files:", path)

In [ ]:
train = pd.read_csv(os.path.join(path,'mitbih_train.csv'), header = None)
test = pd.read_csv(os.path.join(path,'mitbih_test.csv'), header = None)

train.head()

CNN-1D

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
pip install tensorflow

In [2]:
X_train = train.iloc[:, :-1].values
y_train = train.iloc[:, -1].values

X_test = test.iloc[:, :-1].values
y_test = test.iloc[:, -1].values



NameError: name 'train' is not defined

Normalizing ECG Signals

In [ ]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test =  scaler.transform(X_test)

Data Leakage if we do fit on X_test. X is used inside the model so only that is scaled not y

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_train = X_train[...,np.newaxis]
X_test = X_test[..., np.newaxis]


In [ ]:
X_train.shape

One Hot Encoding is done to maintain equality amog labels 0 to 4. 4 is not greater than 0

In [3]:

from tensorflow.keras.utils import to_categorical

num_classes = len(np.unique(y_train))

y_train= to_categorical(y_train, num_classes)
y_test= to_categorical(y_test, num_classes)

c:\Users\shree\anac\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):

KeyboardInterrupt



In [ ]:
y_train.shape

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D , MaxPooling1D , Flatten , Dense , Dropout



In [ ]:
model = Sequential([
    Conv1D(32, kernel_size = 5, activation ="relu", 
           input_shape =(X_train.shape[1], 1)),
    MaxPooling1D(pool_size=2),

    Conv1D(64, kernel_size =5 , activation ="relu"),
    MaxPooling1D(pool_size =2),

    Conv1D(128, kernel_size=3, activation ="relu"),
    Flatten(),

    Dense(128, activation = "relu"),
    Dropout(0.5),

    Dense(num_classes, activation ="softmax")       
           
           
    ])

In [ ]:
model.compile(
    optimizer = "adam",
    loss = "categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
#history = model.fit(
    X_train, y_train,
    epochs=30,
    batch_size = 64,
    validation_split=0.2,
    verbose=1
)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print("Test accuracy:", test_acc)


The Kaggle ECG heartbeat dataset contains pre-segmented and normalized heartbeat signals derived from MIT-BIH. The CNN model was trained on beat-level ECG segments rather than raw continuous ECG recordings.